In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Step 1: Install And Imprort Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import tensorflow as tf
from tensorflow import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, AveragePooling2D,MaxPooling2D,BatchNormalization,Dropout,GlobalAveragePooling2D
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.utils import image_dataset_from_directory
from keras.models import Model
from keras.applications import ResNet101

# Step 2: Read the Dataset

In [ ]:
# Define directories
training_dir = '/kaggle/input/fruits/fruits-360_dataset_100x100/fruits-360/Training'
test_dir = '/kaggle/input/fruits/fruits-360_dataset_100x100/fruits-360/Test'

In [ ]:
# Number of training and test images
image_files = glob(training_dir + '/*/*.jp*g')
test_image_files = glob(test_dir + '/*/*.jp*g')

# Number of classes
folders = glob(training_dir + '/*')
num_classes = len(folders)
print ('Total Classes = ' + str(num_classes))

In [ ]:
test_dir

In [ ]:
training_dir

In [ ]:
# Load datasets using image_dataset_from_directory
training_generator = image_dataset_from_directory(
    training_dir, image_size=(224, 224), batch_size=200, label_mode="categorical"
)

test_generator = image_dataset_from_directory(
    test_dir, image_size=(224, 224), batch_size=200, label_mode="categorical"
)

In [ ]:
# from keras.applications.resnet101 import preprocess_input

# ResNet101 Model Building

In [ ]:
# ResNet101 Model Building
IMAGE_SIZE = [224, 224]
resnet101 = ResNet101(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


for layer in resnet101.layers:    # Freeze ResNet101 layers
    layer.trainable = False

# Add custom layers
x = Flatten()(resnet101.output)
x = Dense(4096, activation='relu')(x)
x = Dense(1096, activation='relu')(x)
x = Dense(num_classes, activation='softmax')(x)

# Build the model
model_resnet101 = Model(inputs=resnet101.input, outputs=x)

# Compile the model
model_resnet101.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Model summary
model_resnet101.summary()

In [ ]:
# Train the model
history_resnet101 = model_resnet101.fit(
    training_generator,
    steps_per_epoch=10000,
    epochs=5,
    validation_data=test_generator,
    validation_steps=3000
)

In [ ]:
# Display results
print('ResNet101 Training Accuracy:', history_resnet101.history['accuracy'])
print('ResNet101 Validation Accuracy:', history_resnet101.history['val_accuracy'])